In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests

import plotly.express as px
from urllib.request import urlopen
import json

import plotly.figure_factory as ff

Define the parameters of the API

In [174]:
data_set = 'http://api.census.gov/data/2021/acs/acs1/spp?get='
for_param = 'county%20'
state = '11,51,24'
key = '0af0fdc7bcf3fef21eb8850a72d58bb373bf4c7c'

## Create variables to pull from census data

[Variable Documentation](https://api.census.gov/data/2021/acs/acs1/spp/variables.html)

In [184]:
# create a list of variables to pull as well as the column name mapping they should contain
variables_mapping = {
    'GEO_ID':'geo_id',
    'COUNTY':'county',
    'STATE':'state',
    'POPGROUP':'race_ethnic_group_code',
    'POPGROUP_LABEL':'population_group_label',
    'S0201_246E':'poverty_rates_for_families',
    'S0201_006E':'total_population',
    'S0201_226E':'income_past_12_months',
    'S0201_099E':'ba_or_higher_degree_earned_25_and_over',
    'S0201_120E':'same_house_1yr_ago',
    'S0201_154E':'employment_status',
    'S0201_168E':'commuting_to_work',
    'S0201_208E':'class_of_worker',
    'S0201_213E':'income_past_yr',
    'S0201_242E':'health_insurance_coverage'
    }
variables = ','.join(variables_mapping.keys())

# url = api_call+variables+',S0201_001E&for='+area+'&in=state:'+state+'&in=county:'+cnty+'&in=tract:'+tract+'&key='+key
url = data_set+variables+'&for='+for_param+'&in=state:11,51,24'+'&key='+key
# url = data_set+variables+'&for=block%20group:*&in=state:11,51,24'+'&key='+key
req = requests.get(url)
json = req.json()
df = pd.DataFrame(json)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.reset_index(inplace=True, drop=True)
df.rename(variables_mapping, axis='columns', inplace=True)
# df.columns = df.columns.str.lower()
print(len(df))
df.head()

146


,geo_id,county,state,race_ethnic_group_code,population_group_label,poverty_rates_for_families,total_population,income_past_12_months,ba_or_higher_degree_earned_25_and_over,same_house_1yr_ago,employment_status,commuting_to_work,class_of_worker,income_past_yr,health_insurance_coverage,state,county
0,0500000US51059,059,51,001,Total population,5.1,1139720,285620,64.9,86.3,907409,594677,593517,410660,1125337,51,059
1,0500000US51059,059,51,002,White alone,2.7,568638,156595,74.2,86.7,468051,304664,301596,233157,559702,51,059
2,0500000US51059,059,51,003,White alone or in combination with one or more...,3.1,708005,185052,70.6,86.8,562082,370544,368086,271020,698013,51,059
3,0500000US51059,059,51,004,Black or African American alone,7.9,108328,23593,50.1,83.5,86906,55660,55043,39617,105503,51,059
4,0500000US51059,059,51,005,Black or African American alone or in combinat...,8.5,136669,27950,50.1,84.0,103671,66108,65649,47300,133672,51,059


In [178]:
# print the length of the geo_id column
len(df.geo_id.loc[1]) # after US is concat of (state, county)

14

# Define Categorizations for the Black/African American population 

* Also define how we will categorize the white population in order to identify potential "wealth gaps"
  * For the time being, I defined "black" as "African" appears (includes population groups such as 'West African' which does not have "Black" in the label) in the race/ethnicity column AND "white" does not appear


In [132]:
print("Number of different populations: ", df.population_group_label.nunique())
print("\nBlack population groups: \n\t", [i for i in list(df.population_group_label.unique()) if ('africa' in i.lower()) and ('white' not in i.lower())])
print("\nWhite population groups: \n\t", [i for i in list(df.population_group_label.unique()) if 'white' in i.lower()])

Number of different populations:  86

Black population groups: 
	 ['Western Africa', 'Black or African American alone or in combination with one or more other races', 'Subsaharan African (All groups)', 'Black or African American in combination with one or more other races', 'Black or African American alone', 'Black or African American alone or in combination with one or more other races, not Hispanic or Latino', 'African', 'Black or African American alone, not Hispanic or Latino', 'Africa']

White population groups: 
	 ['White alone or in combination with one or more other races', 'White; Some other race', 'White; Asian', 'White in combination with one or more other races', 'White alone or in combination with one or more other races, not Hispanic or Latino', 'White; Black or African American', 'White alone', 'White; American Indian and Alaska Native', 'White alone, not Hispanic or Latino']


In [186]:
# create variables indicating whether in white / black population
df.loc[:,'black_pop'] = df.population_group_label.apply(lambda x: 'Black or African American' if ('africa' in x.lower()) and ('white' not in x.lower()) else '')
df.loc[:,'white_pop'] = df.population_group_label.apply(lambda x: 'White' if ('white' in x.lower()) else '')
                                                        

In [188]:
avg_agg = ['poverty_rates_for_families',
           'income_past_12_months',
           'ba_or_higher_degree_earned_25_and_over',
           'same_house_1yr_ago',
           'employment_status',
           'commuting_to_work',
            'class_of_worker',
            'income_past_yr',
            'health_insurance_coverage'
          ]
sum_agg = ['total_population']
# aggregate data by each population and average
df.groupby(['geo_id', 'black_pop'])[avg_agg].mean().reset_index()

/var/folders/g9/n5sfsl5n0z98mvb4hh40ftzr0000gn/T/ipykernel_44567/3515720849.py:13: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  df.groupby(['geo_id', 'black_pop'])[avg_agg].mean().reset_index()


,geo_id,black_pop,income_past_12_months,employment_status,commuting_to_work,class_of_worker,income_past_yr,health_insurance_coverage
0,0500000US11001,,1.170305e+55,5.031248e+60,3.218483e+60,3.230892e+60,2.905138e+60,5.999811e+60
1,0500000US11001,Black or African American,1.431565e+19,5.776356e+22,2.778353e+22,2.865353e+22,3.313029e+22,7.114858e+22
2,0500000US24003,,1.573351e+55,4.738813e+55,3.059402e+55,2.973202e+55,2.250642e+55,5.666294e+55
3,0500000US24003,Black or African American,6.334069e+18,2.076873e+19,1.362890e+19,1.326315e+19,9.854107e+18,2.444753e+21
4,0500000US24005,,1.777984e+65,5.719020e+66,3.472177e+65,3.526994e+65,2.771077e+65,7.018062e+67
5,0500000US24005,Black or African American,1.506341e+19,4.941655e+22,3.237378e+22,3.288353e+22,2.379903e+21,6.246307e+22
6,0500000US24031,,9.560719e+145,3.011466e+155,1.930290e+148,1.955726e+148,1.387129e+146,3.736684e+158
7,0500000US24031,Black or African American,7.952587e+28,2.597853e+32,1.663935e+31,1.682185e+31,1.236964e+29,3.246420e+32
8,0500000US24033,,1.292425e+86,4.510506e+95,2.792974e+87,2.815321e+87,2.036044e+86,5.561789e+97
9,0500000US24033,Black or African American,2.791443e+27,9.465870e+27,5.841666e+27,5.903726e+27,4.558685e+27,1.135231e+28


# Next Steps

1. Figure out how to get the API data in a more granular level